In [122]:
import numpy as np

GRID_SIZE = 5
NUM_PICKUP_LOCATIONS = 3
NUM_DROPOFF_LOCATIONS = 3
MAX_BLOCKS_PER_LOCATION = 5
NUM_AGENTS = 3
REWARD_PICKUP_DROPOFF = 13
COST_EMPTY_SPACE = -1
NUM_ITERATIONS = 500

learningRate = 0.3 # alpha
discountFactor = 0.5 # gamma

pickupLocations = [(4, 0, 5), (3, 1, 5), (1, 4, 5)] # x,y coord and num blocks
dropoffLocations = [(0, 0, 0), (0, 2, 0), (4, 3, 0)] # x,y coord and num blocks
occupiedLocations = [(2,2),(2,4),(2,0)]


class Agent:
    #current state is a x,y location for the agent
    currentState = [0,0]
    holdingBlock = False
    # this agent's specific q-table (starts as all zeros)
    qTable = np.zeros((GRID_SIZE,GRID_SIZE,4))

    def __init__(self, startState, holdingBlock):
        self.currentState = startState
        self.holdingBlock = holdingBlock
    
    def updateLoc(self,x,y):
        self.currentState[0] = x
        self.currentState[1] = y

redAgent = Agent([2,2], False)
blueAgent = Agent([2,4], False)
blackAgent = Agent([2,0], False)



def random_action():
    return np.random.randint(4) # 0: up, 1: down, 2: left, 3: right

def isValidStep(currentStep, nextAction):
    isValid = True
    # checks for overflow of array bounds and if the proposed next step would overlap with another agent's current position
    if (nextAction == 0 and currentStep[1] == 0) or (nextAction == 0 and [currentStep[0],currentStep[1]-1] in occupiedLocations):
        isValid = False
        print("broke in validation 1")
    if (nextAction == 1 and currentStep[1] == 4) or (nextAction == 1 and [currentStep[0],currentStep[1]+1] in occupiedLocations):
        isValid = False
        print("broke in validation 2")
    if (nextAction == 2 and currentStep[0] == 0) or (nextAction == 2 and [currentStep[0]-1,currentStep[1]] in occupiedLocations):
        isValid = False
        print("broke in validatio 3n")
    if (nextAction == 3 and currentStep[0] == 4) or (nextAction == 3 and [currentStep[0]+1,currentStep[1]] in occupiedLocations):
        isValid = False
        print("broke in validation 4")

    print("{} {}".format(currentStep, nextAction))

    return isValid
    
def pTrainingStep(agent):
    startState = [-100,-100]
    nextState = [-110, -110]
    startState[0] = agent.currentState[0]
    startState[1] = agent.currentState[1]
    
    randAction = random_action()
    print(randAction)
    nextState[0] = startState[0]
    nextState[1] = startState[1]

    valid = False
    while not valid:
        print("start state: {}".format(startState))
        if randAction == 0 and isValidStep(startState, randAction):
            # next move is up
            nextState[1] -= 1
            valid = True
            print("validated move from {} to {}".format(startState, nextState))
        elif randAction == 1 and isValidStep(startState, randAction):
            # next move is down
            nextState[1] += 1
            valid = True
            print("validated move from {} to {}".format(startState, nextState))
        elif randAction == 2 and isValidStep(startState, randAction):
            # next move is left
            nextState[0] -=-1
            valid = True
            print("validated move from {} to {}".format(startState, nextState))
        elif randAction == 3 and isValidStep(startState, randAction):
            # next move is right
            nextState[0] += 1
            valid = True
            print("validated move from {} to {}".format(startState, nextState))
        else:
            # invalid something generate a new action
            valid = False
            print("invalidated move from {} to {} -- need to generate a different one".format(startState,nextState))
            randAction = random_action()

    print("moved an agent from {} to {}".format(startState, nextState))

    nextStateBestQ = np.max(agent.qTable[nextState[0]][nextState[1]])

    if(nextState in pickupLocations or nextState in dropoffLocations):
        reward = 13
    else:
        reward = -1

    qTableEdit = learningRate * (reward + discountFactor * nextStateBestQ - agent.qTable[startState[0]][startState[1]][randAction])
    print("q table orig: {}".format(agent.qTable[startState[0]][startState[1]][randAction]))
    print("q table edit: {}\n".format(qTableEdit))
    agent.qTable[startState[0]][startState[1]][randAction] += qTableEdit


for i in range(500):
    print("=======  {}th loop ======".format(i))
    print("red move")
    pTrainingStep(redAgent)
    print("blkue move")
    pTrainingStep(blueAgent)
    print("black move")
    pTrainingStep(blackAgent)

for i in range(5):
    for j in range(5):
        print("loc {}x{} = {}".format(i,j,redAgent.qTable[i][j]))


=======  0th loop ======
red move
3
start state: [2, 2]
[2, 2] 3
validated move from [2, 2] to [3, 2]
moved an agent from [2, 2] to [3, 2]
q table orig: 0.0
q table edit: -0.3

blkue move
1
start state: [2, 4]
broke in validation 2
[2, 4] 1
invalidated move from [2, 4] to [2, 4] -- need to generate a different one
start state: [2, 4]
[2, 4] 2
validated move from [2, 4] to [3, 4]
moved an agent from [2, 4] to [3, 4]
q table orig: 0.0
q table edit: -0.3

black move
3
start state: [2, 0]
[2, 0] 3
validated move from [2, 0] to [3, 0]
moved an agent from [2, 0] to [3, 0]
q table orig: 0.0
q table edit: -0.3

=======  1th loop ======
red move
3
start state: [2, 2]
[2, 2] 3
validated move from [2, 2] to [3, 2]
moved an agent from [2, 2] to [3, 2]
q table orig: -0.3
q table edit: -0.21

blkue move
3
start state: [2, 4]
[2, 4] 3
validated move from [2, 4] to [3, 4]
moved an agent from [2, 4] to [3, 4]
q table orig: 0.0
q table edit: -0.3

black move
3
start state: [2, 0]
[2, 0] 3
validated move